<a href="https://colab.research.google.com/github/JuliaLapova/AI-Assistant/blob/main/Faiss_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Подготовка](#scrollTo=RqMbEUHAK03C)

>[Создаем индекс](#scrollTo=DZxi5hl-LC1Z)

>[Пример использования](#scrollTo=t6qb2xgdKuL3)



In [ ]:
from getpass import getpass
OPENAI_API_KEY = getpass('Enter your OpenAI key: ')
#OPENAI_API_KEY = 'your_key'

# Подготовка

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
hack_dir = '/gdrive/MyDrive/ai-hacks'

Mounted at /gdrive


In [ ]:
! pip install -qqq langchain requests openai transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 111.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00


In [ ]:
! pip install -qqq tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.9 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
import requests
import pandas as pd

In [ ]:
import re
import json
import pandas as pd

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [ ]:
# Загрузка библиотек
import pandas as pd
import json

# Загрузка файла CSV с данными
data = pd.read_csv('/gdrive/MyDrive/ai-hacks/ai-hacks2.csv', sep='\t', encoding='cp1251')

# Преобразование в JSON
json_data = data.to_json(orient="records")

# Запись данных в файл JSON
with open('/gdrive/MyDrive/ai-hacks/ai-hacks2.json', 'w') as f:
    f.write(json_data)

In [ ]:
with open('/gdrive/MyDrive/ai-hacks/ai-hacks2.json', 'r', encoding='utf-8') as f:
    hacks_qa = json.load(f)

hack_df = pd.DataFrame(hacks_qa)
hack_df = hack_df.drop(columns=["Unnamed: 4"])
display(hack_df.head())

,QUESTION,ANSWER,SOURCE,CATEGORY
0,Предоставлять дополнительную меру социальной п...,Заявителем является родитель (усыновитель) реб...,< ID_Вопроса_в_БД: 1 > Предоставлять дополните...,Детская карта
1,Предоставлять дополнительную меру социальной п...,"ребенок - гражданин РФ, имеющий место жительст...",< ID_Вопроса_в_БД: 2 > Предоставлять дополните...,Детская карта
2,Предоставлять дополнительную меру социальной п...,При наличии в семье двух и более детей единовр...,< ID_Вопроса_в_БД: 3 > Предоставлять дополните...,Детская карта
3,Предоставлять дополнительную меру социальной п...,Единовременная компенсационная выплата при рож...,< ID_Вопроса_в_БД: 4 > Предоставлять дополните...,Детская карта
4,Предоставлять дополнительную меру социальной п...,"Гражданам, дети которых находятся на полном го...",< ID_Вопроса_в_БД: 5 > Предоставлять дополните...,Детская карта


In [ ]:
hack_df.sample(4)

,QUESTION,ANSWER,SOURCE,CATEGORY
397,Предоставлять меры социальной поддержки выпуск...,отсутствие у заявителя права на предоставление...,< ID_Вопроса_в_БД: 398 > Предоставлять меры со...,"Сироты, денежное пособие сиротам, покупка одежды"
55,Предоставлять меру социальной поддержки детям-...,Да. Документы (сведения) о регистрации ребенка...,< ID_Вопроса_в_БД: 56 > Предоставлять меру соц...,None
363,"Зачислять в образовательные организации, реали...","Документ, удостоверяющий личность заявителя:\n...",< ID_Вопроса_в_БД: 364 > Зачислять в образоват...,"Школа, первый класс"
394,Предоставлять меры социальной поддержки выпуск...,"документ, удостоверяющий личность доверенного ...",< ID_Вопроса_в_БД: 395 > Предоставлять меры со...,"Сироты, денежное пособие сиротам, покупка одежды"


# Создаем индекс

In [ ]:
sources = [Document(page_content=' '.join([str(item) if item is not None else '' for item in row.tolist()[:2]]), metadata={"source": row.tolist()[2]}) for i, row in hack_df.iterrows()]

In [ ]:
for source in sources:
  print(type(source), source.metadata)
  break

<class 'langchain.schema.document.Document'> {'source': '< ID_Вопроса_в_БД: 1 > Предоставлять дополнительную меру социальной поддержки в виде единовременной компенсационной выплаты при рождении ребенка (усыновлении в возрасте до шести месяцев) для приобретения предметов детского ассортимента и продуктов детского питания'}


In [ ]:
#@markdown faiss index creating

source_chunks = []
splitter = CharacterTextSplitter(separator=" ", chunk_size=512, chunk_overlap=0)
for source in sources:
    for chunk in splitter.split_text(source.page_content):
         source_chunks.append(Document(page_content=chunk, metadata=source.metadata))

search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings())

In [ ]:
#@markdown faiss index saving
search_index.save_local("/gdrive/MyDrive/ai-hacks/")

In [ ]:
#@markdown copy faiss index to gdrive
# ! cp "/gdrive/MyDrive/ai-hacks/" $hack_dir -r

In [ ]:
search_index = FAISS.load_local(folder_path="/gdrive/MyDrive/ai-hacks/", embeddings=OpenAIEmbeddings())

In [ ]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0.1, max_tokens=512))

def clean_question(q):
    q = q.replace("меня", "человека")\
    .replace("мне", "человеку")\
    .replace("я", "человек")\
    .replace("мной", "человеком")
    return q

def edit_answer(a):
    try:
        for splitter in ['SOURCES: ', 'SOURCES:\n', 'SOURCES:']:
            if splitter in a:
                break
        a, sources = a.split(splitter)
        sources = sources\
        .replace('\n\n', ',')\
        .replace('\n', ',')\
        .replace('[', '')\
        .replace(']', '')\
        .split(',')
        if len(sources) == 0:
            return a, ['https://www.google.com',]
        elif type(sources) is str:
            return a, [sources,]
        return a, sources
    except Exception as e:
        a, sources = a, ['https://www.google.com',]
        print(e)
        return a, sources

def get_answer(question, verbose=False):
    answer = (
        chain(
            {
                "input_documents": search_index.similarity_search(question, k=3),
                "question": clean_question(question),
            },
            return_only_outputs=False,
        )["output_text"]
    )
    print(answer) if verbose else None
    answer, sources = edit_answer(answer)
    return answer, sources

def print_answer(question, verbose=False):
    answer, sources = get_answer(question, verbose=verbose)
    print("\033[095mОТВЕТ: \033[0m", answer)
    print("\n\033[095mИСТОЧНИКИ: \033[0m")
    for s in sources:
        print(s)

# edit_answer(test)

# Пример использования

In [ ]:
question = "Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при передаче ребенка на воспитание в семью_Какой результат предоставления услуги?"
c = search_index.similarity_search(question, k=4)

In [ ]:
[s.metadata['source'] for s in c]

['< ID_Вопроса_в_БД: 80 > Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при передаче ребенка на воспитание в семью',
 '< ID_Вопроса_в_БД: 88 > Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при передаче ребенка на воспитание в семью',
 '< ID_Вопроса_в_БД: 87 > Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при передаче ребенка на воспитание в семью',
 '< ID_Вопроса_в_БД: 78 > Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при передаче ребенка на воспитание в семью']

In [ ]:
import openai

def remove_weird_tags(text):
    cleaned_text = re.sub(r'[\xa0]', ' ', text)
    return cleaned_text

def extract_sources(context):
    metadata = [s.metadata['source'] for s in context]
    sources = []
    for s in metadata:
        if type(s) is list:
            for ss in s:
                if ss not in sources:
                    sources.append(ss)
        elif type(s) is str:
            if s not in sources:
                sources.append(s)
    return sources

def get_context(question):
    context = search_index.similarity_search(question, k=3)
    sources = extract_sources(context)
    context = [remove_weird_tags(d.page_content) for d in context if len(d.page_content) > 20]
    context = '\n\n###\n\n'.join(context)
    return context, sources

def answer_with_openai(question, verbose=False):
    context, sources = get_context(question)
    print(context) if verbose else None
    answer = answer_with_context(question, context)
    print(answer) if verbose else None
    print(sources) if verbose else None
    return answer, sources

def print_openai_answer(question, verbose=False):
    answer, sources = answer_with_openai(question, verbose=verbose)
    print("\033[095mОТВЕТ: \033[0m", answer)
    print("\n\033[095mИСТОЧНИКИ: \033[0m")
    for s in sources:
        print(s)

def answer_with_context(
    question,
    context,
    model="gpt-4", #    model="gpt-3.5-turbo",
    max_tokens=2400,
    stop_sequence=None
):
    task = "Answer the question based on the context below, you can think on English but return answer in Russian\n\n"
    info = f"Context: {context}\n\n---\n\nQuestion: {question}\nAnswer:"
    messages = [{"role": "system", "content": f"{task}{info}"}]
    try:
        # Create a completions using the question and context
        completion = openai.ChatCompletion.create(
            messages=messages,
            temperature=0.02,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            model=model,
        )
        return completion['choices'][0]['message']['content'].strip()
    except Exception as e:
        print("[ERROR]", e)
        return ""

In [ ]:
%%time
print_openai_answer('Какой результат предоставления услуги по предоставлению мер социальной поддержки в виде единовременного пособия при передаче ребенка на воспитание в семью?', verbose=True)

Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при передаче ребенка на воспитание в семью_При передачи нескольких детей в семью на кого оформлять обращение? При передаче на воспитание в семью двух и более детей пособие выплачивается на каждого ребенка.

###

Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при передаче ребенка на воспитание в семью_Какой результат предоставления услуги? 1. Принятие решения о назначении единовременного пособия при передаче ребенка на воспитание в семью.
2. Принятие решения администрации района об отказе в предоставлении государственной услуги по назначению единовременного пособия при передаче ребенка в семью.
3. Перечисление денежных средств на счет заявителя или посредством почтовой

###

Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при передаче ребенка на воспитание в семью_Какие основные обстоятельства для оформления 

In [ ]:
%%time
print_answer("Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при передаче ребенка на воспитание в семью_Какой результат предоставления услуги?", verbose=True)

 Результатом предоставления услуги является принятие решения о назначении единовременного пособия при передаче ребенка на воспитание в семью или принятие решения администрации района об отказе в предоставлении государственной услуги по назначению единовременного пособия при передаче ребенка в семью или перечисление денежных средств на счет заявителя или посредством почтовой.
SOURCES: Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при пере
ОТВЕТ:   Результатом предоставления услуги является принятие решения о назначении единовременного пособия при передаче ребенка на воспитание в семью или принятие решения администрации района об отказе в предоставлении государственной услуги по назначению единовременного пособия при передаче ребенка в семью или перечисление денежных средств на счет заявителя или посредством почтовой.


ИСТОЧНИКИ: 
Прием документов для предоставления мер социальной поддержки в виде единовременного пособия при пере
CPU times: 

In [ ]:
%%time
print_openai_answer('Как получить загран паспорт? ', verbose=True)

период оформления паспорта.

###

действительный заграничный паспорт гражданина РФ (при наличии). 5. Квитанция об уплате государственной пошлины за выдачу паспорта в размере 300 рублей. 6. При обнаружении неточности или ошибочности записей, произведенных в паспорте, подлежащем замене, дополнительно предоставляется документ, содержащий верные сведения. При этом в заявлении указываются верные сведения, в соответствии с представленными документами.
Для получения заграничного паспорта необходимо предоставить действующий заграничный паспорт, если он есть. Также нужно оплатить государственную пошлину за выдачу паспорта в размере 300 рублей. Если в паспорте, который подлежит замене, обнаружены неточности или ошибки, необходимо предоставить дополнительный документ, содержащий верные сведения. В заявлении указываются верные сведения, в соответствии с представленными документами.
['< ID_Вопроса_в_БД: 759 > Государственная услуга по выдаче, замене паспортов гражданина Российской Федерации, удосто

In [ ]:
%%time
print_openai_answer('Какой срок предоставления бесплатного проезда на междугородном транспорте к месту лечения и обратно? ', verbose=True)

виде обеспечения проезда за счет средств бюджета Санкт-Петербурга в размере полной стоимости проезда к месту лечения и обратно предоставляется без ограничения количества поездок при наличии медицинских показаний.

Дополнительная мера социальной поддержки по обеспечению проезда к месту отдыха и обратно за счет средств бюджета Санкт-Петербурга предоставляется один раз в год.

###

Государственная услуга предоставления Фондом социального страхования Российской Федерации гражданам, имеющим право на получение государственной социальной помощи в виде набора социальных услуг, государственной услуги по предоставлению при наличии медицинских показаний путевок на санаторно-курортное лечение, осуществляемое в целях профилактики основных заболеваний, и бесплатного проезда на междугородном транспорте к месту лечения и обратно_Какой срок предоставления? не позднее 10 дней с момента поступления

###

компенсация Денежная компенсация предоставляется в пределах следующих норм расходов на обеспечение пр